In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from pandas import read_csv

In [56]:
import json
import argparse
import psycopg2

In [57]:
import os
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Sequential
from tensorflow.keras.regularizers import l2
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from tensorflow.keras import regularizers
from tensorflow.keras import layers, initializers
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import LSTM, Dense, Dropout, Masking, Embedding
from tensorflow.keras.models import load_model
from tensorflow.keras.initializers import RandomUniform
from array import *
import os.path
import joblib
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, auc, accuracy_score, roc_auc_score,f1_score,log_loss,\
classification_report, roc_curve
import matplotlib.pyplot as plt
from math import sqrt
from sys import argv #Module for receiving parameters from the command line
import io
from PIL import Image
import base64

In [58]:
%matplotlib qt

In [69]:
import sys
sys.path.insert(0, 'modules')
from Config_module import Config
global_config = Config()

# Загрузка параметров

In [60]:
load_params_from_config_file = True #Загрузка параметров из файла
load_params_from_command_line = False #Загрузка параметров из командной строки
args = None

try:
    parser = argparse.ArgumentParser()
    _ = parser.add_argument('--config_file', dest='config_file', action='store_true', help='Load config from file')
    _ = parser.add_argument('--config_path', help='Path to config file: /app/cfg.json')
    _ = parser.add_argument('--cmd_config', dest='cmd_config', action='store_true', help='Load config from cmd line')
    _ = parser.add_argument('--task_id')
    _ = parser.add_argument('--data_path')
    _ = parser.add_argument('--scaler_path')
    _ = parser.add_argument('--neural_path')
    _ = parser.add_argument('--new_model_flag')
    _ = parser.add_argument('--learning_rate')
    _ = parser.add_argument('--epochs')
    _ = parser.add_argument('--steps_per_epoch')
    _ = parser.add_argument('--validation_steps')
    args, unknown = parser.parse_known_args()
    
    if args.config_file:
        load_params_from_config_file = True
        load_params_from_command_line = False
    
    if args.cmd_config:
            load_params_from_config_file = False
            load_params_from_command_line = True
except:
    print("Ошибка парсинга параметров из командной строки")

In [61]:
if load_params_from_config_file:
    #Если есть параметры командной строки
    if args:
        #Если указан путь к конфигу
        if args.config_path:
            with open(config_path, 'r', encoding='utf_8') as cfg:
                temp_data=cfg.read()
        else:
            with open('app/configs/10m/edu_neural.json', 'r', encoding='utf_8') as cfg:
                temp_data=cfg.read()

    # parse file`
    config = json.loads(temp_data)
    
    task_id = str(config['task_id'])
    #Путь для загрузки генерируемых данных
    data_path = config['data_path'] #Путь должен быть без чёрточки в конце
    #Путь для сохранения скалера
    scaler_path = config['scaler_path'] #Путь должен быть без чёрточки в конце
    #Путь для сохранения нейронных сетей
    neural_path = config['neural_path'] #Путь должен быть без чёрточки в конце
    #Флаг необходимости подготовки новой модели (False - дообучение существующей)
    new_model_flag = bool(config['new_model_flag'])
    learning_rate = config['learning_rate']
    epochs = config['epochs']
    steps_per_epoch = config['steps_per_epoch']
    validation_steps = config['validation_steps']
    
if load_params_from_command_line:
    task_id = str(args.task_id)
    data_path = str(args.data_path)
    scaler_path = str(args.scaler_path)
    neural_path = str(args.neural_path) 
    new_model_flag = bool(args.new_model_flag)
    learning_rate = float(args.learning_rate) 
    epochs = int(args.epochs) 
    steps_per_epoch = int(args.steps_per_epoch) 
    validation_steps = str(args.validation_steps) 

Y_shift = 0

In [62]:
type = 'current'# Тип нейросети в ансамбле
period = '1d'

dataset_type = 'num_logic'
dataset_timeframe = '1d_1w'

#data_type_flag = False;
#data_type_flag = 'float16';
data_type_flag = 'float32';
#data_type_flag = 'float64';

#Флаг необходимости масштабирования данных
scale_flag = True



#Флаг тестирования модели
test_model_flag = False

#Флаг необходимости сохранения модели
save_model_flag = True

dataset = dataset_type + '_' + dataset_timeframe

In [8]:

print("Сохранённый датасет отсутствует")
# Импортируем данные для обучения и тестирования
print("Импортируем данные")
if data_type_flag == 'float16':
    init_data_train = pd.read_csv('./'+data_path+'/'+dataset+'_train.csv', dtype = 'float16', sep = ',')
elif data_type_flag == 'float32':
    init_data_train = pd.read_csv('./'+data_path+'/'+dataset+'_train.csv', dtype = 'float32', sep = ',')
else:
    init_data_train = pd.read_csv('./'+data_path+'/'+dataset+'_train.csv', sep = ',')
if data_type_flag == 'float16':
    init_data_test = pd.read_csv('./'+data_path+'/'+dataset+'_test.csv', dtype = 'float16', sep = ',')
elif data_type_flag == 'float32':
    init_data_test = pd.read_csv('./'+data_path+'/'+dataset+'_test.csv', dtype = 'float32', sep = ',')
else:
    init_data_test = pd.read_csv('./app/data/'+dataset+'_test.csv', sep = ',')
print("Доля NaN данных в датасете train:", init_data_train.isna().sum() / init_data_train.shape[0]*100)
print("Доля NaN данных в датасете test:", init_data_test.isna().sum() / init_data_test.shape[0]*100)

#Исключаем nan и inf
init_data_train.replace([np.inf, -np.inf], np.nan).dropna(axis=1)
init_data_test.replace([np.inf, -np.inf], np.nan).dropna(axis=1)

# Устанавливаем размерность датасетов
n_train = init_data_train.shape[0]
p_train = init_data_train.shape[1]
print("Число факторов: ", p_train)
n_test = init_data_test.shape[0]
p_test = init_data_test.shape[1]
# Формируем данные в numpy-массив
init_data_train = init_data_train.values
init_data_test = init_data_test.values
# Подготовка данных для обучения и тестирования (проверки)
print("Подготавливаем обучающие, тестовые и предиктивные выборки")
train_start = 0
train_end = n_train
test_start = 0
test_end = n_test
data_train = init_data_train[np.arange(train_start, train_end), :]
data_test = init_data_test[np.arange(test_start, test_end), :]
#Выбор данных
print("Выбираем данные")
trainX = data_train[:, 3:]
trainY = data_train[:, 2]
train_quotes_close = data_train[:, 1]
testX = data_test[:, 3:]
testY = data_test[:, 2]
test_quotes_close = data_test[:, 1]
# Масштабирование данных
print("Масштабируем данные")
x_scaler = MinMaxScaler(feature_range=(-1, 1))
y_scaler = MinMaxScaler(feature_range=(-1, 1))
if scale_flag: 
    x_scaler.fit(trainX)
    scaler_filename = './'+scaler_path+'/scaler_'+dataset+'.save'
    joblib.dump(x_scaler, scaler_filename) 
#Изменяем размерность массива, для обеспечения возможности масштабирования Y
trainY = trainY.reshape(-1, 1)
testY = testY.reshape(-1, 1)
train_quotes_close = train_quotes_close.reshape(-1, 1)
test_quotes_close = test_quotes_close.reshape(-1, 1)
if scale_flag:
    #y_scaler.fit(trainY)
    trainX = x_scaler.transform(trainX)
    testX = x_scaler.transform(testX)
#Изменяем размерность массива Х, для рекурентной нейросети
trainX = np.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
testX = np.reshape(testX, (testX.shape[0], 1, testX.shape[1]))

Сохранённый датасет отсутствует
Импортируем данные
Доля NaN данных в датасете train: Datetime     0.0
Close        0.0
Y            0.0
Open:5m      0.0
High:5m      0.0
            ... 
gap_14:5m    0.0
gap_16:5m    0.0
gap_18:5m    0.0
gap_20:5m    0.0
gap_40:5m    0.0
Length: 2662, dtype: float64
Доля NaN данных в датасете test: Datetime     0.0
Close        0.0
Y            0.0
Open:5m      0.0
High:5m      0.0
            ... 
gap_14:5m    0.0
gap_16:5m    0.0
gap_18:5m    0.0
gap_20:5m    0.0
gap_40:5m    0.0
Length: 2662, dtype: float64
Число факторов:  2662
Подготавливаем обучающие, тестовые и предиктивные выборки
Выбираем данные
Масштабируем данные


In [9]:
#Проверяем число анализируемых факторов
print("Число анализируемых факторов", trainX.shape[2])
print("Число анализируемых данных тренировочной выборки", trainX.shape[0])
print("Число анализируемых данных тестовой выборки", testX.shape[0])

Число анализируемых факторов 2659
Число анализируемых данных тренировочной выборки 147817
Число анализируемых данных тестовой выборки 16426


In [51]:
def plt_to_png(graph):
    buffer = io.BytesIO()
    graph.savefig(buffer, format='png')
    buffer.seek(0)
    image_png = buffer.getvalue()
    buffer.close()
    graphic = base64.b64encode(image_png)
    graphic = graphic.decode('utf-8')
    graph.close()

    return graphic

In [10]:
factors_count = trainX.shape[2]
data_count = trainX.shape[0]

In [11]:
def data_train():
    while True:
        x = trainX
        y = trainY
        yield (x,y)

In [12]:
def data_test():
    while True:
        x = testX
        y = testY
        yield (x,y)

In [13]:
def tfdata_generator(x_datas, y_datas, is_training, batch_size=128):
    '''Construct a data generator using `tf.Dataset`. '''

    def map_fn(x_data, y_data):
        '''Preprocess raw data to trainable input. '''
        x = x_data 
        y = y_data
        return x, y

    dataset = tf.data.Dataset.from_tensor_slices((x_datas, y_datas))
    
    if is_training:
        dataset = dataset.shuffle(1000)  # depends on sample size
        dataset = dataset.map(map_fn)
        dataset = dataset.batch(batch_size)
        dataset = dataset.repeat()
        dataset = dataset.prefetch(tf.data.AUTOTUNE)

    return dataset

In [14]:
training_set = tfdata_generator(trainX, trainY,is_training=True)

train_generator = data_train()
valid_generator = data_test()

In [15]:
checkpoint_filepath = 'tmp/checkpoint'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)

In [16]:
from datetime import datetime

logdir = "logs/scalars/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=logdir)

In [17]:
es = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

In [18]:
#Проверяем существование нейронной сети
file_path = './'+neural_path+'/ansamble_'+dataset+'_v1.h5'

In [19]:
#Тестируем нейронную сеть
if (os.access(file_path, os.F_OK) == True) & (test_model_flag == True):
    print("Тестируем нейронную сеть")
    #Загружаем нейронную сеть
    print("Загружаем сеть")
    model = load_model('./'+neural_path+'/ansamble_'+dataset+'_v1.h5');

In [20]:
#Дообучаем нейроннуюю сеть
if (os.access(file_path, os.F_OK) == True) & (test_model_flag == False) & (new_model_flag == False):
    print("Дообучаем нейронную сеть")
    #Загружаем нейронную сеть
    print("Загружаем сеть")
    model = load_model('./'+neural_path+'/ansamble_'+dataset+'_v1.h5');
    
    #Обучаем нейронную сеть
    print("Обучаем нейронную сеть")
    his = model.fit(
        training_set, 
        validation_data=valid_generator, 
        epochs=epochs,
        steps_per_epoch=steps_per_epoch, 
        validation_steps = validation_steps, 
        callbacks=[
            #model_checkpoint_callback,
            #es
        ]
    )
    
    if save_model_flag == True:    
        #Сохраняем нейронную сеть
        print("Сохраняем нейронную сеть")
        model.save('./'+neural_path+'/ansamble_'+dataset+'_v1.h5')

In [21]:
if ((os.access(file_path, os.F_OK) == False) | (new_model_flag == True)) & (test_model_flag == False) :
    print("Нейронная сеть Отсутствует")
    # define and fit the final model
    print("Формируем модель нейросети")
    model = Sequential()
    model.add(Dropout(0.2, input_shape=(trainX.shape[1], trainX.shape[2])))
    
    #num_logic start
    if dataset_type == 'num_logic':
        model.add(Dense(
            1000,
            activation='relu', 
            kernel_regularizer=regularizers.l2(0.001),
            kernel_initializer='random_normal',
            bias_initializer='zeros'
        ))   
        model.add(Dense(
            500,
            activation='relu', 
            kernel_regularizer=regularizers.l2(0.001),
            kernel_initializer='random_normal',
            bias_initializer='zeros'
        ))
        model.add(Dense(
            units=250, 
            #125,
            activation='tanh', 
            kernel_regularizer=regularizers.l2(0.001),
            kernel_initializer='random_normal',
            bias_initializer='zeros'
        ))
        model.add(Dense(
            units=150, 
            #75,
            activation='relu', 
            kernel_regularizer=regularizers.l2(0.001),
            kernel_initializer='random_normal',
            bias_initializer='zeros'
        ))
    #num_logic end

    model.add(Dense(
        3, 
        kernel_regularizer=regularizers.l2(0.01),
        kernel_initializer='random_normal',
        bias_initializer='zeros',
        activation='softmax'
    ))

    opt =  keras.optimizers.Adam(clipvalue=1., clipnorm=1., learning_rate = learning_rate,amsgrad = True)
    print("Компилируем нейронную сеть")
    model.compile(loss='sparse_categorical_crossentropy', optimizer=opt, metrics=['accuracy'])	   
    
    #Обучаем нейронную сеть
    print("Обучаем нейронную сеть")
    his = model.fit(
        training_set, 
        validation_data=valid_generator, 
        epochs=epochs,
        steps_per_epoch=steps_per_epoch, 
        validation_steps = validation_steps, 
        callbacks=[
            #model_checkpoint_ca|llback,
            #es
            #tensorboard_callback
        ]
    )

    if save_model_flag == True:    
        #Сохраняем нейронную сеть
        print("Сохраняем нейронную сеть")
        model.save('./'+neural_path+'/ansamble_'+dataset+'_v1.h5')

    #Наблюдаем показатели точности
    fig, ax = plt.subplots()
    ax.set_title('Показатели точности обучения нейронной сети, loss MSE')
    plt.plot(his.history['loss'], label='loss тренировочной выборки')
    plt.plot(his.history['val_loss'], label='loss тестовой выборки')
    plt.legend(loc="upper left")
    losses_results = plt_to_png(plt)
    plt.close()
    #plt.show()

Нейронная сеть Отсутствует
Формируем модель нейросети
Компилируем нейронную сеть
Обучаем нейронную сеть
Epoch 1/350
128/128 [==============================] - 114s 892ms/step - loss: 9.3482 - accuracy: 0.3578 - val_loss: 9.2497 - val_accuracy: 0.3540
Epoch 2/350
128/128 [==============================] - 117s 922ms/step - loss: 9.1895 - accuracy: 0.4455 - val_loss: 9.1319 - val_accuracy: 0.3512
Epoch 3/350
128/128 [==============================] - 38s 296ms/step - loss: 9.0610 - accuracy: 0.4341 - val_loss: 8.8818 - val_accuracy: 0.6208
Epoch 4/350
128/128 [==============================] - 37s 290ms/step - loss: 8.1362 - accuracy: 0.9672 - val_loss: 9.2449 - val_accuracy: 0.6197
Epoch 5/350
128/128 [==============================] - 37s 290ms/step - loss: 8.4749 - accuracy: 0.7860 - val_loss: 8.7965 - val_accuracy: 0.6197
Epoch 6/350
128/128 [==============================] - 37s 289ms/step - loss: 8.6106 - accuracy: 0.5051 - val_loss: 8.6074 - val_accuracy: 0.4585
Epoch 7/350
128/12

128/128 [==============================] - 37s 293ms/step - loss: 4.6091 - accuracy: 0.4697 - val_loss: 4.3553 - val_accuracy: 0.6227
Epoch 57/350
128/128 [==============================] - 37s 293ms/step - loss: 4.5788 - accuracy: 0.4458 - val_loss: 4.2920 - val_accuracy: 0.6103
Epoch 58/350
128/128 [==============================] - 37s 294ms/step - loss: 3.8199 - accuracy: 0.9106 - val_loss: 4.8275 - val_accuracy: 0.6197
Epoch 59/350
128/128 [==============================] - 37s 294ms/step - loss: 3.9527 - accuracy: 0.8228 - val_loss: 4.3050 - val_accuracy: 0.6197
Epoch 60/350
128/128 [==============================] - 37s 294ms/step - loss: 4.2415 - accuracy: 0.5830 - val_loss: 4.1818 - val_accuracy: 0.5629
Epoch 61/350
128/128 [==============================] - 37s 295ms/step - loss: 4.4072 - accuracy: 0.4489 - val_loss: 4.1419 - val_accuracy: 0.5657
Epoch 62/350
128/128 [==============================] - 37s 294ms/step - loss: 4.3561 - accuracy: 0.4502 - val_loss: 4.1223 - val_a

Epoch 112/350
128/128 [==============================] - 38s 299ms/step - loss: 2.4621 - accuracy: 0.8427 - val_loss: 3.3050 - val_accuracy: 0.6197
Epoch 113/350
128/128 [==============================] - 38s 299ms/step - loss: 2.3959 - accuracy: 0.8322 - val_loss: 2.6937 - val_accuracy: 0.6546
Epoch 114/350
128/128 [==============================] - 38s 299ms/step - loss: 2.6796 - accuracy: 0.6863 - val_loss: 2.7022 - val_accuracy: 0.6479
Epoch 115/350
128/128 [==============================] - 38s 298ms/step - loss: 2.9616 - accuracy: 0.5081 - val_loss: 2.6870 - val_accuracy: 0.6563
Epoch 116/350
128/128 [==============================] - 38s 299ms/step - loss: 2.9444 - accuracy: 0.5003 - val_loss: 2.6875 - val_accuracy: 0.6014
Epoch 117/350
128/128 [==============================] - 38s 297ms/step - loss: 2.5382 - accuracy: 0.7087 - val_loss: 2.6292 - val_accuracy: 0.6474
Epoch 118/350
128/128 [==============================] - 38s 298ms/step - loss: 2.9140 - accuracy: 0.5074 - val_

128/128 [==============================] - 38s 301ms/step - loss: 1.7944 - accuracy: 0.8419 - val_loss: 2.1060 - val_accuracy: 0.6732
Epoch 168/350
128/128 [==============================] - 38s 301ms/step - loss: 1.9904 - accuracy: 0.7574 - val_loss: 2.1087 - val_accuracy: 0.6508
Epoch 169/350
128/128 [==============================] - 38s 301ms/step - loss: 2.3847 - accuracy: 0.5162 - val_loss: 2.1050 - val_accuracy: 0.6591
Epoch 170/350
128/128 [==============================] - 38s 301ms/step - loss: 2.3721 - accuracy: 0.5158 - val_loss: 2.1038 - val_accuracy: 0.6487
Epoch 171/350
128/128 [==============================] - 38s 302ms/step - loss: 1.9823 - accuracy: 0.7257 - val_loss: 2.0997 - val_accuracy: 0.6750
Epoch 172/350
128/128 [==============================] - 38s 301ms/step - loss: 2.3534 - accuracy: 0.5225 - val_loss: 2.0824 - val_accuracy: 0.6607
Epoch 173/350
128/128 [==============================] - 38s 301ms/step - loss: 2.3286 - accuracy: 0.5482 - val_loss: 2.0661 -

128/128 [==============================] - 38s 301ms/step - loss: 1.5454 - accuracy: 0.8185 - val_loss: 1.7926 - val_accuracy: 0.6632
Epoch 223/350
128/128 [==============================] - 38s 301ms/step - loss: 2.0659 - accuracy: 0.5200 - val_loss: 1.8016 - val_accuracy: 0.6564
Epoch 224/350
128/128 [==============================] - 38s 301ms/step - loss: 2.0505 - accuracy: 0.5286 - val_loss: 1.7742 - val_accuracy: 0.6560
Epoch 225/350
128/128 [==============================] - 38s 301ms/step - loss: 1.7005 - accuracy: 0.7148 - val_loss: 1.7899 - val_accuracy: 0.6624
Epoch 226/350
128/128 [==============================] - 38s 301ms/step - loss: 1.9971 - accuracy: 0.5487 - val_loss: 1.7751 - val_accuracy: 0.6488
Epoch 227/350
128/128 [==============================] - 38s 301ms/step - loss: 2.0065 - accuracy: 0.5605 - val_loss: 1.7892 - val_accuracy: 0.6579
Epoch 228/350
128/128 [==============================] - 38s 301ms/step - loss: 2.0408 - accuracy: 0.5187 - val_loss: 1.7687 -

128/128 [==============================] - 38s 301ms/step - loss: 1.8597 - accuracy: 0.5179 - val_loss: 1.6125 - val_accuracy: 0.6590
Epoch 278/350
128/128 [==============================] - 38s 301ms/step - loss: 1.8429 - accuracy: 0.5294 - val_loss: 1.5780 - val_accuracy: 0.6711
Epoch 279/350
128/128 [==============================] - 38s 301ms/step - loss: 1.6206 - accuracy: 0.6575 - val_loss: 1.5774 - val_accuracy: 0.6667
Epoch 280/350
128/128 [==============================] - 38s 302ms/step - loss: 1.6601 - accuracy: 0.6189 - val_loss: 1.5616 - val_accuracy: 0.6564
Epoch 281/350
128/128 [==============================] - 38s 302ms/step - loss: 1.8175 - accuracy: 0.5489 - val_loss: 1.5602 - val_accuracy: 0.6727
Epoch 282/350
128/128 [==============================] - 38s 301ms/step - loss: 1.8184 - accuracy: 0.5439 - val_loss: 1.5915 - val_accuracy: 0.6634
Epoch 283/350
128/128 [==============================] - 38s 301ms/step - loss: 1.6142 - accuracy: 0.6642 - val_loss: 1.9475 -

128/128 [==============================] - 38s 303ms/step - loss: 1.6944 - accuracy: 0.5325 - val_loss: 1.4666 - val_accuracy: 0.6473
Epoch 333/350
128/128 [==============================] - 38s 302ms/step - loss: 1.5979 - accuracy: 0.5951 - val_loss: 1.4265 - val_accuracy: 0.6734
Epoch 334/350
128/128 [==============================] - 38s 302ms/step - loss: 1.3918 - accuracy: 0.6882 - val_loss: 1.4190 - val_accuracy: 0.6641
Epoch 335/350
128/128 [==============================] - 39s 304ms/step - loss: 1.6835 - accuracy: 0.5341 - val_loss: 1.4288 - val_accuracy: 0.6574
Epoch 336/350
128/128 [==============================] - 38s 302ms/step - loss: 1.6540 - accuracy: 0.5673 - val_loss: 1.4335 - val_accuracy: 0.6616
Epoch 337/350
128/128 [==============================] - 38s 302ms/step - loss: 1.5768 - accuracy: 0.6054 - val_loss: 1.4324 - val_accuracy: 0.6674
Epoch 338/350
128/128 [==============================] - 38s 302ms/step - loss: 0.8978 - accuracy: 0.9697 - val_loss: 2.1310 -

In [22]:
print("Предсказываем результат")
predict_testY = model.predict(testX, verbose = 1)
predict_trainY = model.predict(trainX, verbose = 1)

Предсказываем результат
4620/4620 [==============================] - 17s 4ms/step


In [23]:
#Преобразовываем выходные сигналы тренировочной выборки
result_predict_trainY = []

for predict in predict_trainY:
    result_predict_trainY.append(np.argmax(predict))
        
result_predict_trainY = np.array(result_predict_trainY)

In [24]:
#Преобразовываем выходные сигналы тестовой выборки
result_predict_testY = []

for predict in predict_testY:
    result_predict_testY.append(np.argmax(predict))
        
result_predict_testY = np.array(result_predict_testY)

In [25]:
# fig, ax = plt.subplots()
# ax.set_title('Сигналы тренировочной выборки')

# y = trainY#Реальные значения
# y1 = result_predict_trainY#Расчетные значения
# y1 = y1+3

# plt.plot(y, label='Размеченые данные')
# plt.plot(y1, label='Расчётные данные нейронной сети')
# plt.title('Тренировочная выборка')
# plt.legend(loc="upper left")
# plt.show()

In [26]:
# fig, ax = plt.subplots()
# ax.set_title('Сигналы тестовой выборки')

# y = testY#Реальные значения
# y1 = result_predict_testY#Расчетные значения
# y1 = y1+3

# plt.plot(y, label='Размеченые данные')
# plt.plot(y1, label='Расчётные данные нейронной сети')
# plt.title('Тестовая выборка')
# plt.legend(loc="upper left")
# plt.show()

# Расчёт трендов

In [27]:
#Расчёт трендов для тренировочной выборки на основе сигналов по разметке
last_train_signal = 2
train_trends_origin = array('f', []) #Массив ожидаемых данных по тренду
for i in range(trainY.shape[0]):
    if trainY[i] != last_train_signal and (trainY[i] == 2 or trainY[i] == 0):
        last_train_signal = trainY[i]
    train_trends_origin.insert(i,last_train_signal)

In [28]:
#Расчёт трендов для тестовой выборки на основе расчётных сигналов
last_test_signal = 2
test_trends_origin = array('f', []) #Массив ожидаемых данных по тренду
for i in range(testY.shape[0]):
    if testY[i] != last_test_signal and (testY[i] == 2 or testY[i] == 0):
        last_test_signal = testY[i]
    test_trends_origin.insert(i,last_test_signal)

In [29]:
#Расчёт трендов для тренировочной выборки на основе расчётных данных
last_train_signal = 2
train_trends_predict = array('f', []) #Массив ожидаемых данных по тренду
for i in range(len(result_predict_trainY)):
    if result_predict_trainY[i] != last_train_signal and (result_predict_trainY[i] == 2 or result_predict_trainY[i] == 0):
        last_train_signal = result_predict_trainY[i]
    train_trends_predict.insert(i,last_train_signal)

In [30]:
#Расчёт трендов для тестовой выборки на основе расчётных сигналов
last_test_signal = 2
test_trends_predict = array('f', []) #Массив ожидаемых данных по тренду
for i in range(len(result_predict_testY)):
    if result_predict_testY[i] != last_test_signal and (result_predict_testY[i] == 2 or result_predict_testY[i] == 0):
        last_test_signal = result_predict_testY[i]
    test_trends_predict.insert(i,last_test_signal)

In [31]:
train_trends_origin = np.asarray(train_trends_origin).astype(int)
test_trends_origin = np.asarray(test_trends_origin).astype(int)
train_trends_predict = np.asarray(train_trends_predict).astype(int)
test_trends_predict = np.asarray(test_trends_predict).astype(int)

# Расчёт показателей точности

In [73]:
train_accuracy_score = accuracy_score(train_trends_origin, train_trends_predict)
train_roc_auc_score = roc_auc_score(train_trends_origin, train_trends_predict)
train_precision_score = precision_score(train_trends_origin, train_trends_predict, pos_label=2)
train_recall_score = recall_score(train_trends_origin, train_trends_predict, pos_label=2)
train_f1_score = f1_score(train_trends_origin, train_trends_predict, pos_label=2)
train_log_loss = log_loss(train_trends_origin, train_trends_predict)

In [32]:
#Выводим данные результатов анализа точности
print("РЕЗУЛЬТАТЫ АНАЛИЗА ТОЧНОСТИ");

print("ТРЕНИРОВОЧНАЯ ВЫБОРКА")
print('accuracy:', accuracy_score(train_trends_origin, train_trends_predict))
print('roc-auc:', roc_auc_score(train_trends_origin, train_trends_predict))
print('precision:', precision_score(train_trends_origin, train_trends_predict, pos_label=2))
print('recall:', recall_score(train_trends_origin, train_trends_predict, pos_label=2))
print('f1:', f1_score(train_trends_origin, train_trends_predict, pos_label=2))
print('logloss:', log_loss(train_trends_origin, train_trends_predict))

РЕЗУЛЬТАТЫ АНАЛИЗА ТОЧНОСТИ
ТРЕНИРОВОЧНАЯ ВЫБОРКА
accuracy: 0.5894247616985868
roc-auc: 0.588884209626579
precision: 0.5843793985005601
recall: 0.5616786305908338
f1: 0.572804189602016
logloss: 14.180922885443248


In [74]:
test_accuracy_score = accuracy_score(test_trends_origin, test_trends_predict)
test_roc_auc_score = roc_auc_score(test_trends_origin, test_trends_predict)
test_precision_score = precision_score(test_trends_origin, test_trends_predict, pos_label=2)
test_recall_score = recall_score(test_trends_origin, test_trends_predict, pos_label=2)
test_f1_score = f1_score(test_trends_origin, test_trends_predict, pos_label=2)
test_log_loss = log_loss(test_trends_origin, test_trends_predict)

In [33]:
print("ТЕСТОВАЯ ВЫБОРКА")
print('accuracy:', accuracy_score(test_trends_origin, test_trends_predict))
print('roc-auc:', roc_auc_score(test_trends_origin, test_trends_predict))
print('precision:', precision_score(test_trends_origin, test_trends_predict, pos_label=2))
print('recall:', recall_score(test_trends_origin, test_trends_predict, pos_label=2))
print('f1:', f1_score(test_trends_origin, test_trends_predict, pos_label=2))
print('logloss:', log_loss(test_trends_origin, test_trends_predict))

ТЕСТОВАЯ ВЫБОРКА
accuracy: 0.570437111895775
roc-auc: 0.5959170204899599
precision: 0.47232882246010904
recall: 0.7186539643515673
f1: 0.570018281535649
logloss: 14.836830886342538


# Расчёт доходности

# РАСЧЕТ ДОХОДНОСТИ ТРЕНИРОВОЧНОЙ ВЫБОРКИ

In [34]:
print("РАСЧЕТ ДОХОДНОСТИ ТРЕНИРОВОЧНОЙ ВЫБОРКИ")
profit_origin_arr = array('f', [])#Массив оригинальной доходности
profit_origin_arr_shift = array('f', [])#Массив оригинальной доходности со смещением на 1 день
profit_calc_sigma_arr = array('f', [])#Массив доходности по стандартному отклонению

open_pos_flag_origin = False
open_pos_flag_calc = False

open_price_origin = 0 #Цена открытия позиции
open_price_origin_shift = 0 #Цена открытия позиции со смещением на 1 день
open_price_calc = 0 #Цена открытия позиции

profit_origin = 0 #Текущая доходность волны
profit_origin_shift = 0 #Текущая доходность волны со смещением на 1 день
profit_calc = 0 #Текущая доходность волны

total_profit_origin = 1 #Общая доходность
total_profit_origin_shift = 1 #Общая доходность со смещением на 1 день
total_profit_calc_sigma = 1 #Общая доходность по стандартному отклонению

count_profit_origin = 0 #Номер рассчитанной доходности
count_profit_calc_sigma = 0 #Номер рассчитанной доходности по стандартному отклонению

one_profit_origin = 1 #Общая доходность при торговле одной акцией
one_profit_origin_shift = 1 #Общая доходность со смещением на 1 день при торговле одной акцией
one_profit_calc_sigma = 1 #Общая доходность по стандартному отклонению при торговле одной акцией

РАСЧЕТ ДОХОДНОСТИ ТРЕНИРОВОЧНОЙ ВЫБОРКИ


In [35]:
#Рассчитываем доходность по уровням
for i in range(trainX.shape[0]):
    #Опредеяем доходность по размеченным данным
    if trainY[i] == 2 and open_pos_flag_origin == False:
        open_pos_flag_origin = True #Открываем позицию
        open_price_origin = train_quotes_close[i] #Фиксируем открытие позиции
        try:
            open_price_origin_shift = train_quotes_close[i+1] #Фиксируем открытие позиции
        except:
            open_price_origin_shift = train_quotes_close[i] #Фиксируем открытие позиции
    if trainY[i] == 0 and open_pos_flag_origin == True:
        open_pos_flag_origin = False #Закрываем позицию
        profit_origin = train_quotes_close[i]/open_price_origin #Фиксируем прибыль
        one_profit_origin = one_profit_origin + (profit_origin-1) #Вычисляем доходность на одну акцию
        try:
            profit_origin_shift = train_quotes_close[i+1]/open_price_origin_shift #Фиксируем прибыль со смещением на 1 день
        except:
            profit_origin_shift = train_quotes_close[i]/open_price_origin_shift
        total_profit_origin = total_profit_origin * profit_origin #Рассчитываем общую доходность
        one_profit_origin_shift = one_profit_origin_shift + (profit_origin_shift-1) #Вычисляем доходность на одну акцию
        total_profit_origin_shift = total_profit_origin_shift * profit_origin_shift #Рассчитываем общую доходность со смещением на 1 день
        profit_origin_arr.insert(count_profit_origin, profit_origin) #Добавляем прибыль в массив
        profit_origin_arr_shift.insert(count_profit_origin, profit_origin_shift) #Добавляем прибыль со смещением на 1 день в массив
        count_profit_origin = count_profit_origin+1 #Делаем инкримент счетчика доходности

#Обнуляем данные
open_pos_flag_calc = False
open_price_calc = 0 #Цена открытия позиции
profit_calc = 0 #Текущая доходность волны

#Рассчитываем доходность тренировочной выборки
for i in range(trainX.shape[0]):
    #Опредеяем доходность по рассчетным данным
    if result_predict_trainY[i] == 2 and open_pos_flag_calc == False:
        open_pos_flag_calc = True #Открываем позицию
        open_price_calc = train_quotes_close[i] #Фиксируем открытие позиции
    if result_predict_trainY[i] == 0 and open_pos_flag_calc == True:
        open_pos_flag_calc = False #Закрываем позицию
        profit_calc = train_quotes_close[i]/open_price_calc #Фиксируем прибыль
        one_profit_calc_sigma = one_profit_calc_sigma + (profit_calc-1) #Вычисляем доходность на одну акцию
        total_profit_calc_sigma = total_profit_calc_sigma * profit_calc #Рассчитываем общую доходность
        profit_calc_sigma_arr.insert(count_profit_calc_sigma, profit_calc) #Добавляем прибыль в массив
        count_profit_calc_sigma = count_profit_calc_sigma+1 #Делаем инкримент счетчика доходности


result_profit_origin_arr = np.asarray(profit_origin_arr)
result_profit_origin_arr_shift = np.asarray(profit_origin_arr_shift)
result_profit_calc_sigma_arr = np.asarray(profit_calc_sigma_arr)

print("Накопленная доходность по размеченным данным: ", total_profit_origin)
print("Накопленная доходность по размеченным данным со смещением на 1 день: ", total_profit_origin_shift)
print("Накопленная доходность по расчётным данным по стандартному отклонению: ", total_profit_calc_sigma)

print("")

print("Доходность на одну акцию размеченным данным: ", one_profit_origin)
print("Доходность на одну акцию по размеченным данным со смещением на 1 день: ", one_profit_origin_shift)
print("Доходность на одну акцию по расчётным данным по стандартному отклонению: ", one_profit_calc_sigma)

Накопленная доходность по размеченным данным:  [9.2040325e+21]
Накопленная доходность по размеченным данным со смещением на 1 день:  [1.1367858e+12]
Накопленная доходность по расчётным данным по стандартному отклонению:  [3079040.2]

Доходность на одну акцию размеченным данным:  [108.98194]
Доходность на одну акцию по размеченным данным со смещением на 1 день:  [85.47331]
Доходность на одну акцию по расчётным данным по стандартному отклонению:  [121.49428]


In [36]:
fig, ax = plt.subplots()
ax.set_title('Доходность трейдов тренировочной выборки')

ax.hlines(1, 0, count_profit_origin)
ax.hlines(0.5, 0, count_profit_origin)
ax.hlines(0, 0, count_profit_calc_sigma)

y_calc_profit_origin = result_profit_origin_arr
y_calc_profit_origin_shift = result_profit_origin_arr_shift - 0.5
y_calc_profit_sigma = result_profit_calc_sigma_arr - 1
plt.plot(y_calc_profit_origin, label='Размеченные данные')
plt.plot(y_calc_profit_origin_shift, label='Размеченные данные со смещением на 1 день')
plt.plot(y_calc_profit_sigma, label='Рассчётные данные данные')
plt.legend(loc="upper left")
plt.show()

In [37]:
profit_origin_arr_summ = array('f', [])#Массив накопленной оригинальной доходности
profit_origin_arr_shift_summ = array('f', [])#Массив накопленной оригинальной доходности со смещением на 1 день
profit_calc_sigma_arr_summ = array('f', [])#Массив накопленной доходности по стандартному отклонению

profit_origin_arr_summ_show = array('f', [])#Массив смещённой накопленной оригинальной доходности
profit_origin_arr_shift_summ_show = array('f', [])#Массив смещённой накопленной оригинальной доходности со смещением на 1 день
profit_calc_sigma_arr_summ_show = array('f', [])#Массив смещённой накопленной доходности по стандартному отклонению

last_profit_origin = 1
last_profit_origin_shift = 1
last_profit_calc_sigma = 1

shift_origin = 0
shift_origin_shift = -1
shift_calc_sigma = -2

for i in range(len(profit_origin_arr)):
    profit_origin_arr_summ_show.insert(i,last_profit_origin*profit_origin_arr[i]-shift_origin)
    profit_origin_arr_summ.insert(i,last_profit_origin*profit_origin_arr[i])
    last_profit_origin = last_profit_origin*profit_origin_arr[i]
	
for i in range(len(profit_origin_arr_shift)):
    profit_origin_arr_shift_summ_show.insert(i,last_profit_origin_shift*profit_origin_arr_shift[i]-shift_origin_shift)
    profit_origin_arr_shift_summ.insert(i,last_profit_origin_shift*profit_origin_arr_shift[i])
    last_profit_origin_shift = last_profit_origin_shift*profit_origin_arr_shift[i]

for i in range(len(profit_calc_sigma_arr)):
    profit_calc_sigma_arr_summ_show.insert(i,last_profit_calc_sigma*profit_calc_sigma_arr[i]-shift_calc_sigma)
    profit_calc_sigma_arr_summ.insert(i,last_profit_calc_sigma*profit_calc_sigma_arr[i])
    last_profit_calc_sigma = last_profit_calc_sigma*profit_calc_sigma_arr[i]

In [38]:
fig, ax = plt.subplots()
ax.set_title('Накопленная доходность тренировочной выборки')

y_calc_profit_origin = profit_origin_arr_summ_show
y_calc_profit_origin_shift = profit_origin_arr_shift_summ_show
y_calc_profit_sigma = profit_calc_sigma_arr_summ_show
plt.plot(y_calc_profit_origin, label='Размеченные данные')
plt.plot(y_calc_profit_origin_shift, label='Размеченные данные со смещением')
plt.plot(y_calc_profit_sigma, label='Рассчётные данные')
plt.legend(loc="upper left")
plt.show()

In [39]:
train_profit_origin_arr = profit_origin_arr#Массив оригинальной доходности
train_profit_origin_arr_shift = profit_origin_arr_shift#Массив оригинальной доходности со смещением на 1 день
train_profit_calc_sigma_arr = profit_calc_sigma_arr#Массив доходности по стандартному отклонению

train_profit_origin_arr_summ = profit_origin_arr_summ#Массив накопленной оригинальной доходности
train_profit_origin_arr_shift_summ = profit_origin_arr_shift_summ#Массив накопленной оригинальной доходности со смещением на 1 день
train_profit_calc_sigma_arr_summ = profit_calc_sigma_arr_summ#Массив накопленной доходности по стандартному отклонению

# Тестовая выборка

In [40]:
	#РАСЧЕТ ДОХОДНОСТИ ТЕСТОВОЙ ВЫБОРКИ
print("")
print("РАСЧЕТ ДОХОДНОСТИ ТЕСТОВОЙ ВЫБОРКИ")
profit_origin_arr = array('f', [])#Массив оригинальной доходности
profit_origin_arr_shift = array('f', [])#Массив оригинальной доходности со смещением на 1 день
profit_calc_sigma_arr = array('f', [])#Массив доходности по стандартному отклонению

open_pos_flag_origin = False
open_pos_flag_calc = False

open_price_origin = 0 #Цена открытия позиции
open_price_origin_shift = 0 #Цена открытия позиции со смещением на 1 день
open_price_calc = 0 #Цена открытия позиции

profit_origin = 0 #Текущая доходность волны
profit_origin_shift = 0 #Текущая доходность волны со смещением на 1 день
profit_calc = 0 #Текущая доходность волны

total_profit_origin = 1 #Общая доходность
total_profit_origin_shift = 1 #Общая доходность со смещением на 1 день
total_profit_calc_sigma = 1 #Общая доходность по стандартному отклонению

count_profit_origin = 0 #Номер рассчитанной доходности
count_profit_calc_sigma = 0 #Номер рассчитанной доходности по стандартному отклонению

one_profit_origin = 1 #Общая доходность при торговле одной акцией
one_profit_origin_shift = 1 #Общая доходность со смещением на 1 день при торговле одной акцией
one_profit_calc_sigma = 1 #Общая доходность по стандартному отклонению при торговле одной акцией

#Рассчитываем доходность по уровням
for i in range(testX.shape[0]):
    #Опредеяем доходность по размеченным данным
    if testY[i] == 2 and open_pos_flag_origin == False:
        open_pos_flag_origin = True #Открываем позицию
        open_price_origin = test_quotes_close[i] #Фиксируем открытие позиции
        open_price_origin_shift = test_quotes_close[i+1] #Фиксируем открытие позиции
    if testY[i] == 0 and open_pos_flag_origin == True:
        open_pos_flag_origin = False #Закрываем позицию
        profit_origin = test_quotes_close[i]/open_price_origin #Фиксируем прибыль
        one_profit_origin = one_profit_origin + (profit_origin-1) #Вычисляем доходность на одну акцию
        profit_origin_shift = test_quotes_close[i+1]/open_price_origin_shift #Фиксируем прибыль со смещением на 1 день
        total_profit_origin = total_profit_origin * profit_origin #Рассчитываем общую доходность
        one_profit_origin_shift = one_profit_origin_shift + (profit_origin_shift-1) #Вычисляем доходность на одну акцию
        total_profit_origin_shift = total_profit_origin_shift * profit_origin_shift #Рассчитываем общую доходность со смещением на 1 день
        profit_origin_arr.insert(count_profit_origin, profit_origin) #Добавляем прибыль в массив
        profit_origin_arr_shift.insert(count_profit_origin, profit_origin_shift) #Добавляем прибыль со смещением на 1 день в массив
        count_profit_origin = count_profit_origin+1 #Делаем инкримент счетчика доходности

#Обнуляем данные
open_pos_flag_calc = False
open_price_calc = 0 #Цена открытия позиции
profit_calc = 0 #Текущая доходность волны

#Рассчитываем доходность тренировочной выборки
for i in range(testX.shape[0]):
    #Опредеяем доходность по рассчетным данным
    if result_predict_testY[i] == 2 and open_pos_flag_calc == False:
        open_pos_flag_calc = True #Открываем позицию
        open_price_calc = test_quotes_close[i] #Фиксируем открытие позиции
    if result_predict_testY[i] == 0 and open_pos_flag_calc == True:
        open_pos_flag_calc = False #Закрываем позицию
        profit_calc = test_quotes_close[i]/open_price_calc #Фиксируем прибыль
        one_profit_calc_sigma = one_profit_calc_sigma + (profit_calc-1) #Вычисляем доходность на одну акцию
        total_profit_calc_sigma = total_profit_calc_sigma * profit_calc #Рассчитываем общую доходность
        profit_calc_sigma_arr.insert(count_profit_calc_sigma, profit_calc) #Добавляем прибыль в массив
        count_profit_calc_sigma = count_profit_calc_sigma+1 #Делаем инкримент счетчика доходности


result_profit_origin_arr = np.asarray(profit_origin_arr)
result_profit_origin_arr_shift = np.asarray(profit_origin_arr_shift)
result_profit_calc_sigma_arr = np.asarray(profit_calc_sigma_arr)

print("Накопленная доходность по размеченным данным: ", total_profit_origin)
print("Накопленная доходность по размеченным данным со смещением на 1 день: ", total_profit_origin_shift)
print("Накопленная доходность по расчётным данным: ", total_profit_calc_sigma)

print("")

print("Доходность на одну акцию размеченным данным: ", one_profit_origin)
print("Доходность на одну акцию по размеченным данным со смещением на 1 день: ", one_profit_origin_shift)
print("Доходность на одну акцию по расчётным данным: ", one_profit_calc_sigma)


РАСЧЕТ ДОХОДНОСТИ ТЕСТОВОЙ ВЫБОРКИ
Накопленная доходность по размеченным данным:  [9.74166e+09]
Накопленная доходность по размеченным данным со смещением на 1 день:  [8.128909e+08]
Накопленная доходность по расчётным данным:  [2.2186478e+09]

Доходность на одну акцию размеченным данным:  [153672.81]
Доходность на одну акцию по размеченным данным со смещением на 1 день:  [153345.72]
Доходность на одну акцию по расчётным данным:  [150971.11]


In [41]:
fig, ax = plt.subplots()
ax.set_title('Доходность трейдов тестовой выборки')

ax.hlines(1, 0, count_profit_origin)
ax.hlines(0.5, 0, count_profit_origin)
ax.hlines(0, 0, count_profit_calc_sigma)

y_calc_profit_origin = result_profit_origin_arr
y_calc_profit_origin_shift = result_profit_origin_arr_shift - 0.5
y_calc_profit_sigma = result_profit_calc_sigma_arr - 1
plt.plot(y_calc_profit_origin, label='Размеченные данные')
plt.plot(y_calc_profit_origin_shift, label='Размеченные данные со смещением на 1 день')
plt.plot(y_calc_profit_sigma, label='Расчётные данные')
plt.legend(loc="upper left")
plt.show()

In [42]:
profit_origin_arr_summ = array('f', [])#Массив накопленной оригинальной доходности
profit_origin_arr_shift_summ = array('f', [])#Массив накопленной оригинальной доходности со смещением на 1 день
profit_calc_sigma_arr_summ = array('f', [])#Массив накопленной доходности по стандартному отклонению

profit_origin_arr_summ_show = array('f', [])#Массив смещённой накопленной оригинальной доходности
profit_origin_arr_shift_summ_show = array('f', [])#Массив смещённой накопленной оригинальной доходности со смещением на 1 день
profit_calc_sigma_arr_summ_show = array('f', [])#Массив смещённой накопленной доходности по стандартному отклонению

last_profit_origin = 1
last_profit_origin_shift = 1
last_profit_calc_sigma = 1

shift_origin = 0
shift_origin_shift = -1
shift_calc_sigma = -2

for i in range(len(profit_origin_arr)):
    profit_origin_arr_summ_show.insert(i,last_profit_origin*profit_origin_arr[i]-shift_origin)
    profit_origin_arr_summ.insert(i,last_profit_origin*profit_origin_arr[i])
    last_profit_origin = last_profit_origin*profit_origin_arr[i]
	
for i in range(len(profit_origin_arr_shift)):
    profit_origin_arr_shift_summ_show.insert(i,last_profit_origin_shift*profit_origin_arr_shift[i]-shift_origin_shift)
    profit_origin_arr_shift_summ.insert(i,last_profit_origin_shift*profit_origin_arr_shift[i])
    last_profit_origin_shift = last_profit_origin_shift*profit_origin_arr_shift[i]

for i in range(len(profit_calc_sigma_arr)):
    profit_calc_sigma_arr_summ_show.insert(i,last_profit_calc_sigma*profit_calc_sigma_arr[i]-shift_calc_sigma)
    profit_calc_sigma_arr_summ.insert(i,last_profit_calc_sigma*profit_calc_sigma_arr[i])
    last_profit_calc_sigma = last_profit_calc_sigma*profit_calc_sigma_arr[i]

In [43]:
fig, ax = plt.subplots()
ax.set_title('Накопленная доходность тестовой выборки')

y_calc_profit_origin = profit_origin_arr_summ_show
y_calc_profit_origin_shift = profit_origin_arr_shift_summ_show
y_calc_profit_sigma = profit_calc_sigma_arr_summ_show
plt.plot(y_calc_profit_origin, label='Размеченные данные')
plt.plot(y_calc_profit_origin_shift, label='Размеченные данные со смещением на 1 день')
plt.plot(y_calc_profit_sigma, label='Рассчётные сигналы')
plt.legend(loc="upper left")
plt.show()

test_profit_origin_arr = profit_origin_arr#Массив оригинальной доходности
test_profit_origin_arr_shift = profit_origin_arr_shift#Массив оригинальной доходности со смещением на 1 день
test_profit_calc_sigma_arr = profit_calc_sigma_arr#Массив доходности по стандартному отклонению

test_profit_origin_arr_summ = profit_origin_arr_summ#Массив накопленной оригинальной доходности
test_profit_origin_arr_shift_summ = profit_origin_arr_shift_summ#Массив накопленной оригинальной доходности со смещением на 1 день
test_profit_calc_sigma_arr_summ = profit_calc_sigma_arr_summ#Массив накопленной доходности по стандартному отклонению

# РАСЧЕТ РИСКОВ

In [44]:
print("РАСЧЕТ РИСКОВ")

#ТРЕНИРОВОЧНАЯ ВЫБОРКА
print("ТРЕНИРОВОЧНАЯ ВЫБОРКА")

#Дисперсия портфеля

#Дисперсия по размеченным данным
print("Дисперсия по размеченным данным: ", np.var(train_profit_origin_arr_summ))

#Дисперсия по размеченным данным со смещением
print("Дисперсия по размеченным данным со смещением: ", np.var(train_profit_origin_arr_shift_summ))

#Дисперсия по расчётным данным
print("Дисперсия по расчётным данным: ", np.var(train_profit_calc_sigma_arr_summ))

#Коэффициент Шарпа
R = 5#Безрисковая ставка
#Коэффициент Шарпа по размеченным данным
print("Коэффициент Шарпа по размеченным данным: ", ((train_profit_origin_arr_summ[len(train_profit_origin_arr_summ)-1]-1)*100-R)/sqrt(np.var(train_profit_origin_arr_summ)))

#Коэффициент Шарпа по размеченным данным со смещением
print("Коэффициент Шарпа по размеченным данным со смещением: ", ((train_profit_origin_arr_shift_summ[len(train_profit_origin_arr_shift_summ)-1]-1)*100-R)/sqrt(np.var(train_profit_origin_arr_shift_summ)))

sharp = 0		
if train_profit_calc_sigma_arr_summ != 0:
    sharp = ((train_profit_calc_sigma_arr_summ[len(train_profit_calc_sigma_arr_summ)-1]-1)*100-R)/sqrt(np.var(train_profit_calc_sigma_arr_summ))

#Коэффициент Шарпа по расчётным данным
print("Коэффициент Шарпа по расчётным данным: ", sharp)

#ТЕСТОВАЯ ВЫБОРКА
print("ТЕСТОВАЯ ВЫБОРКА")
#Дисперсия портфеля

#Дисперсия по размеченным данным
print("Дисперсия по размеченным данным: ", np.var(test_profit_origin_arr_summ))

#Дисперсия по размеченным данным со смещением
print("Дисперсия по размеченным данным со смещением: ", np.var(test_profit_origin_arr_shift_summ))

#Дисперсия по расчётным данным
print("Дисперсия по расчётным данным: ", np.var(test_profit_calc_sigma_arr_summ))

#Коэффициент Шарпа
R = 5#Безрисковая ставка
		#Коэффициент Шарпа по размеченным данным
sharp = 0		
if sqrt(np.var(test_profit_origin_arr_summ)) != 0:
    sharp = ((test_profit_origin_arr_summ[len(test_profit_origin_arr_summ)-1]-1)*100-R)/sqrt(np.var(test_profit_origin_arr_summ))
print("Коэффициент Шарпа по размеченным данным: ", sharp)

#Коэффициент Шарпа по размеченным данным со смещением
sharp = 0		
if sqrt(np.var(test_profit_origin_arr_shift_summ)) != 0:
    sharp = ((test_profit_origin_arr_shift_summ[len(test_profit_origin_arr_shift_summ)-1]-1)*100-R)/sqrt(np.var(test_profit_origin_arr_shift_summ))

#Коэффициент Шарпа по расчётным данным
print("Коэффициент Шарпа по размеченным данным со смещением: ", sharp)

sharp = 0
if sqrt(np.var(test_profit_calc_sigma_arr_summ)) != 0:
    sharp = ((test_profit_calc_sigma_arr_summ[len(test_profit_calc_sigma_arr_summ)-1]-1)*100-R)/sqrt(np.var(test_profit_calc_sigma_arr_summ))

#Коэффициент Шарпа по расчётным данным
print("Коэффициент Шарпа по расчётным данным: ", sharp)

РАСЧЕТ РИСКОВ
ТРЕНИРОВОЧНАЯ ВЫБОРКА
Дисперсия по размеченным данным:  inf
Дисперсия по размеченным данным со смещением:  2.4811296e+22
Дисперсия по расчётным данным:  972141560000.0
Коэффициент Шарпа по размеченным данным:  0.0
Коэффициент Шарпа по размеченным данным со смещением:  721.6957435279093
Коэффициент Шарпа по расчётным данным:  312.28548809084657
ТЕСТОВАЯ ВЫБОРКА
Дисперсия по размеченным данным:  5.412509e+18
Дисперсия по размеченным данным со смещением:  4.3585367e+16
Дисперсия по расчётным данным:  3.9078997e+17
Коэффициент Шарпа по размеченным данным:  418.729718872813
Коэффициент Шарпа по размеченным данным со смещением:  389.36947187588277
Коэффициент Шарпа по расчётным данным:  354.90842775631876


C:\Program Files\Python310\lib\site-packages\numpy\core\_methods.py:236: RuntimeWarning: overflow encountered in multiply
  x = um.multiply(x, x, out=x)


# Сохранение результатов

In [85]:
#Соединение с БД
def connect():
    return psycopg2.connect(
        host=global_config.db_host,
        database=global_config.db_database,
        user=global_config.db_user,
        password=global_config.db_password
    )
conn = connect()

In [89]:
if conn.closed == 1:
    conn = connect()
#Проверяем наличие записи
cur = conn.cursor()
cur.execute("SELECT * FROM public.edu_neural_results WHERE task_id  = %s;", (task_id,))
results = cur.fetchall()
cur.close()

In [96]:
if conn.closed == 1:
    conn = connect()
cur = conn.cursor()
try:
    if len(results) == 0:
        print("Записываем результаты")
        #Записи о результатах в БД нет, записываем новый результат
        cur.execute(
            """
            INSERT INTO public.edu_neural_results 
            (
                task_id, 
                losses_results, 
                train_accuracy_score, 
                train_roc_auc_score, 
                train_precision_score, 
                train_recall_score, 
                train_f1_score, 
                train_log_loss,
                test_accuracy_score, 
                test_roc_auc_score, 
                test_precision_score, 
                test_recall_score, 
                test_f1_score, 
                test_log_loss
            )
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s);
            """,
            (
                task_id, 
                losses_results, 
                train_accuracy_score, 
                train_roc_auc_score, 
                train_precision_score, 
                train_recall_score, 
                train_f1_score, 
                train_log_loss,
                test_accuracy_score, 
                test_roc_auc_score, 
                test_precision_score, 
                test_recall_score, 
                test_f1_score, 
                test_log_loss
            )
        )
    else:
        #Обновляем запись
        print("Обновляем результаты")
        sql = """ UPDATE public.edu_neural_results
                    SET 
                    losses_results = %s, 
                    train_accuracy_score = %s, 
                    train_roc_auc_score = %s, 
                    train_precision_score = %s, 
                    train_recall_score = %s, 
                    train_f1_score = %s, 
                    train_log_loss = %s,
                    test_accuracy_score = %s, 
                    test_roc_auc_score = %s, 
                    test_precision_score = %s, 
                    test_recall_score = %s, 
                    test_f1_score = %s, 
                    test_log_loss = %s
                    WHERE task_id = %s"""
        cur.execute(sql, (
                losses_results, 
                train_accuracy_score, 
                train_roc_auc_score, 
                train_precision_score, 
                train_recall_score, 
                train_f1_score, 
                train_log_loss,
                test_accuracy_score, 
                test_roc_auc_score, 
                test_precision_score, 
                test_recall_score, 
                test_f1_score, 
                test_log_loss,
                task_id
            ))
except Exception as e:
    print("Ошибка записи результатов в БД: ", e)

conn.commit()
cur.close()

Обновляем результаты


In [ ]:
#Обновляем данные по задаче
if conn.closed == 1:
    conn = connect()
cur = conn.cursor()

sql = """ UPDATE data_markup_results
            SET task_status = %s
            WHERE id = %s"""
try:
    cur.execute(sql, ('done', task_id))
except Exception as e:
    print("Ошибка записи информации о закрытии задачи в БД: ", e)